<a href="https://colab.research.google.com/github/sightes/BIGDATA-UBO2022/blob/main/BIGDATA_UBO_19_11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive 
drive.mount("/content/gdrive")
gdrive='content/gdrive/MyDrive/UBO-BIGDATA1'

Mounted at /content/gdrive


In [ ]:
%%capture
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget 'https://dlcdn.apache.org/spark/spark-3.3.1/spark-3.3.1-bin-hadoop3.tgz' -O 'spark-3.3.1-bin-hadoop3.tgz'
!tar -xvf /content/spark-3.3.1-bin-hadoop3.tgz
!pip install -q findspark 

In [ ]:
import os 
os.environ['JAVA_HOME']='/usr/lib/jvm/java-8-openjdk-amd64'
os.environ['SPARK_HOME']='/content/spark-3.3.1-bin-hadoop3'

In [ ]:
import findspark 
findspark.init()
from pyspark.sql import SparkSession
spark=SparkSession.builder.master("local[*]").appName("ejemplo1").getOrCreate()

In [ ]:
!ls

gdrive	sample_data  spark-3.3.1-bin-hadoop3  spark-3.3.1-bin-hadoop3.tgz


In [ ]:
!unzip '/content/gdrive/MyDrive/UBO-BIGDATA1/archive.zip'

Archive:  /content/gdrive/MyDrive/UBO-BIGDATA1/archive.zip
  inflating: 2019-Dec.csv            
  inflating: 2019-Nov.csv            
  inflating: 2019-Oct.csv            
  inflating: 2020-Feb.csv            
  inflating: 2020-Jan.csv            


In [ ]:
df=spark.read.options(header='True',inferSchema='True').csv('/content/*.csv')

In [ ]:
df.count()

20692840

In [ ]:
df.printSchema()

root
 |-- event_time: timestamp (nullable = true)
 |-- event_type: string (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- category_id: long (nullable = true)
 |-- category_code: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- price: double (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- user_session: string (nullable = true)



In [ ]:
df.select('event_type').distinct().show()

+----------------+
|      event_type|
+----------------+
|        purchase|
|            view|
|            cart|
|remove_from_cart|
+----------------+



In [ ]:
df.createOrReplaceTempView("data")
spark.sql("select event_type from data group by event_type limit 5").show()

+----------------+
|      event_type|
+----------------+
|        purchase|
|            view|
|            cart|
|remove_from_cart|
+----------------+



In [ ]:
df.createOrReplaceTempView("data")
spark.sql("select product_id,count(*) from data \
 where event_type='purchase' group by product_id limit 5").show()

In [ ]:
df.select(['product_id']).filter( "event_type='purchase'").distinct().show()



+----------+
|product_id|
+----------+
|   5815137|
|   5670719|
|   5574996|
|   5900643|
|   5863096|
|   5834157|
|   5823267|
|   5868397|
|   5809871|
|   5814871|
|   5885863|
|     59990|
|   5837927|
|   5695639|
|   5767927|
|   5859171|
|   5563822|
|   5744098|
|   5908237|
|   5890174|
+----------+
only showing top 20 rows




ACTIVIDAD


1.   Seleccionar un producto 
2.   identificar las sesiones de carrito 
que tienen ese producto
3.  identificar los productos que pertenecen al mismo carrito que el producto seleccionado


sulloa@ieee.org

 



In [ ]:
spark.sql("select count(distinct(product_id)) from data").show()

+--------------------------+
|count(DISTINCT product_id)|
+--------------------------+
|                     54571|
+--------------------------+



RDD

In [ ]:
spark=SparkSession.builder.getOrCreate()
sc=spark.sparkContext
num=[1,2,3,4,5]

In [ ]:
num_rdd=sc.parallelize(num)

In [ ]:
doble_RDD=num_rdd.map(lambda x:x*2)

In [ ]:
doble_RDD.collect()

[2, 4, 6, 8, 10]

In [ ]:
df.select(['brand']).distinct().show()

+------------+
|       brand|
+------------+
|     beautix|
|     farmona|
|  dr.gloderm|
|   profhenna|
|     philips|
|invisibobble|
|       riche|
|        oniq|
|    lebelage|
|     vilenta|
|       fancy|
|      jaguar|
|      tertio|
|    siberina|
|   koreatida|
|         jas|
|rocknailstar|
|   depilflax|
|protokeratin|
|       essie|
+------------+
only showing top 20 rows



In [ ]:
def myFunc(s):
  if s['brand']=='riche' and s['event_type']== 'cart':
    return [(s['product_id'],1)]
  return []  

In [ ]:
df.createOrReplaceTempView("data")
salida=spark.sql("select product_id,count(*) from data where brand='riche' \
and event_type='cart' group by product_id ")

In [ ]:
lines=df.rdd.flatMap(myFunc).reduceByKey(lambda a,b:a+b)

for e in lines.collect():
  print(e)

(5844571, 20)
(5842234, 5)
(5842215, 10)
(5922110, 3)
(5842235, 12)
(5842217, 11)
(5844573, 3)
(5922112, 2)
(5842219, 9)
(5922114, 1)
(5842258, 20)
(5842220, 6)
(5922115, 1)
(5842202, 17)
(5842221, 9)
(5842240, 12)
(5846097, 5)
(5922097, 1)
(5846098, 21)
(5842203, 13)
(5842222, 25)
(5842241, 6)
(5922117, 6)
(5842204, 17)
(5842223, 12)
(5842242, 7)
(5922118, 9)
(5842224, 9)
(5842205, 12)
(5922119, 2)
(5842206, 2)
(5842225, 6)
(5842226, 6)
(5922121, 1)
(5842227, 11)
(5922103, 1)
(5922122, 1)
(5922104, 2)
(5922123, 1)
(5922124, 1)
(5842268, 131)
(5922106, 3)
(5842212, 3)
(5842231, 16)
(5922107, 5)
(5842213, 12)
(5842232, 20)
(5922108, 1)
(5842214, 13)
(5842233, 4)
(5922109, 1)


In [ ]:
salida.write.mode('overwrite').csv("/content/gdrive/MyDrive/UBO-BIGDATA1/result")